In [1]:
pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

In [4]:
def research(Qid):
    sparql.setQuery("""
    SELECT ?item ?itemLabel
    WHERE 
    {
      ?item wdt:P31 wd:""" + Qid + """. #on pourais rajouté des obligation de référance à des payes etc
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # le label viendra de préférence dans votre langue, et autrement en anglais
      
    }LIMIT 1000
    """)
    #/wdt:P31/wdt:P279
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    mydict = []
    for result in results["results"]["bindings"]:
        # ne prend pas ce ou il y a pas de nom ajouté
        if(result["itemLabel"]["value"][0]!="Q"):                                   
            mydict.append(result["itemLabel"]["value"])
            
    return mydict

In [63]:
listofidqHuman = ['Q22808320', 'Q571', 'Q31629', 'Q188451', 'Q192581', 'Q11424', 'Q11410', 'Q2095', 'Q223557', 'Q3624078']


In [64]:
description = [research(listofidqHuman[0]),research(listofidqHuman[1]),research(listofidqHuman[2]),research(listofidqHuman[3]),research(listofidqHuman[4])
               ,research(listofidqHuman[5]),research(listofidqHuman[6]),research(listofidqHuman[7]),research(listofidqHuman[8]),research(listofidqHuman[9])]

In [7]:
!pip install transformers==3.3.1
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [8]:
model_path = "./generation model/"

In [9]:
tokenizer = T5Tokenizer.from_pretrained(model_path)
model =T5ForConditionalGeneration.from_pretrained(model_path, return_dict=True)

In [10]:
def generate(text,model,tokenizer):
    model.eval()
    input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), return_tensors="pt")  
    outputs = model.generate(input_ids, max_length=300, temperature=0.1)
    return tokenizer.decode(outputs[0])

In [11]:
import unicodedata

In [12]:
import gensim

# Load Google's pre-trained Word2Vec model.
word2vecModel = gensim.models.KeyedVectors.load_word2vec_format('./word2vec model/GoogleNews-vectors-negative300.bin', binary=True) 

In [13]:
#topics contient les sujets pour lesquels il y a des méthodes pour le dialogue
#quand on implémente des nouvelles méthodes il faut rajouter le sujet ici
topics=['music', 'job', 'food', 'game', 'sport']

In [72]:
class Person:
    def __init__(self):
        self.age = random.randint(18, 80)
        self.name = unicodedata.normalize('NFD', random.choice(description[0])).encode('ascii', 'ignore').decode('utf-8')
        self.book = unicodedata.normalize('NFD', random.choice(description[1])).encode('ascii', 'ignore').decode('utf-8')
        self.sport = unicodedata.normalize('NFD', random.choice(description[2])).encode('ascii', 'ignore').decode('utf-8')
        self.music = unicodedata.normalize('NFD', random.choice(description[3])).encode('ascii', 'ignore').decode('utf-8')
        self.job = unicodedata.normalize('NFD', random.choice(description[4])).encode('ascii', 'ignore').decode('utf-8')
        self.film =  unicodedata.normalize('NFD', random.choice(description[5])).encode('ascii', 'ignore').decode('utf-8')
        self.game =  unicodedata.normalize('NFD', random.choice(description[6])).encode('ascii', 'ignore').decode('utf-8')
        self.food =  unicodedata.normalize('NFD', random.choice(description[7])).encode('ascii', 'ignore').decode('utf-8')
        self.objectOfInterest =  unicodedata.normalize('NFD', random.choice(description[8])).encode('ascii', 'ignore').decode('utf-8')
        self.country = unicodedata.normalize('NFD', random.choice(description[9])).encode('ascii', 'ignore').decode('utf-8')

    def generateDescription(self):
        surname=self.name.split(' ')[0]
        string=self.name+' | age | '+str(self.age)+' && '+self.name+' | country | '+self.country
        print(generate(string, model, tokenizer))
        string=surname+' | job | '+self.job
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteSport | '+self.sport
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteMusic | '+self.music
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteGenre | '+self.genre
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteGame | '+self.game
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteFood | '+self.food
        print(generate(string, model, tokenizer))
        string=surname+' | interestedIn | '+self.objectOfInterest
        print(generate(string, model, tokenizer))

    def showProperties(self):
        print('name: '+self.name+'\nage: '+str(self.age)+'\ncountry: '+self.country+'\nbook: '+self.book+'\nmusic: '+self.music+'\njob: '+self.job+'\nsport: '+self.sport+'\nfilm: '+self.film+'\ngame: '+self.game+'\nfood: '+self.food+'\nobject of interest: '+self.objectOfInterest+'\n')
        
    def introduce(self):
        text=self.name.split(' ')[0]+':   '

        #Greeting
        rand=random.randint(0,3)
        if rand==0:
            text+='Hello, '
        elif rand==1:
            text+='Hi, '
        elif rand==2:
            text+='Hey, '
        elif rand==3:
            text+='Good day, '
      
        #Name
        rand=random.randint(0,2)
        if rand==0:
            text+="I'm "
        elif rand==1:
            text+="I am "
        elif rand==2:
            text+="my name is "
        text+=self.name+', '

        #Age
        rand=random.randint(0,1)
        if rand==0:
            text+='I am '
        elif rand==1:
            text+="I'm "
        text+=str(self.age)+' years old '

        #Country
        rand=random.randint(0,3)
        if rand==0:
            text+="and I'm from "
        elif rand==1:
            text+="and I am from "
        elif rand==2:
            text+="and I come from "
        elif rand==3:
            text+="and I live in "
        text+=self.country+'.'

        print(text)

    def tellYourFavouriteFood(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='My favourite food is '
        elif rand==1:
            text+='I like to eat '
        elif rand==2:
            text+='I love '
        elif rand==3:
            text+='My favourite dish is '
        elif rand==4:
            text+='I really enjoy '
        text+=self.food+'.'
        return text

    def startTopicFood(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteFood()+' '
        #Ask the others about their favourite food
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite food?'
        elif rand==1:
            text+='What do you like to eat most?'
        elif rand==2:
            text+='What is your preferred dish?'
        elif rand==3:
            text+='What is your favourite dish?'
        elif rand==4:
            text+='Which food do you enjoy most?'
        print(name+text)
        return text

    def respondToFood(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteFood()
        print(text)

    def tellYourJob(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='I work as a '
        elif rand==1:
            text+='I am '
        elif rand==2:
            text+="I'm "
        elif rand==3:
            text+='My job is '
        elif rand==4:
            text+='I make a living from working as a '
        text+=self.optimizeJob(self.job)+'.'
        return text
    
    def optimizeJob(self, job):
        split=job.split(' ')
        for i in range(len(split)):
            if split[i]=='management':
                split[i]='manager'
            elif split[i]=='support':
                split[i]='supporter'
            elif split[i]=='therapy':
                split[i]='therapist'
            elif split[i]=='supervision':
                split[i]='supervisor'
            elif split[i]=='scouting':
                split[i]='scout'
        optimizedJob=''
        for word in split:
            optimizedJob+=word+' '
        optimizedJob=optimizedJob[:len(optimizedJob)-1]
        return optimizedJob

    def startTopicJob(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourJob()+' '
        #Ask the others about their job
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your job?'
        elif rand==1:
            text+='What are you doing for a living?'
        elif rand==2:
            text+='What are you working?'
        elif rand==3:
            text+='What do you do?'
        elif rand==4:
            text+='What do you work as?'
        print(name+text)
        return text

    def respondToJob(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourJob()
        print(text)

    def tellYourFavouriteMusic(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite music is '
        elif rand==1:
            text+='I like listening to '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='My favourite music genre is '
        elif rand==4:
            text+='I really enjoy listening to '
        elif rand==5:
            text+='I usually listen to '
        text+=self.music+'.'
        return text

    def startTopicMusic(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteMusic()+' '
        #Ask the others about their favourite music
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite music?'
        elif rand==1:
            text+='What do you like to listen to?'
        elif rand==2:
            text+='What is your favourite music genre?'
        elif rand==3:
            text+='What music do you like to listen to?'
        elif rand==4:
            text+='Which type of music do you enjoy most?'
        print(name+text)
        return text

    def respondToMusic(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteMusic()
        print(text)
        
    def tellYourFavouriteGame(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite game is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        text+=self.game+'.'
        return text

    def startTopicGame(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteGame()+' '
        #Ask the others about their favourite game
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite game?'
        elif rand==1:
            text+='What do you like to play?'
        elif rand==2:
            text+='Which games do you play?'
        elif rand==3:
            text+='Are there games that you like to play?'
        elif rand==4:
            text+='Which game do you enjoy most?'
        print(name+text)
        return text

    def respondToGame(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteGame()
        print(text)
        
    def tellYourFavouriteSport(self):
        text=''
        rand=random.randint(0,9)
        if rand==0:
            text+='My favourite sport is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        elif rand==6:
            text+='I like watching '
        elif rand==7:
            text+='I like to watch '
        elif rand==8:
            text+='I\'m a fan of '
        elif rand==9:
            text+='I\'m a '+self.sport+' fan.'
        if not rand==9:
            text+=self.sport+'.'
        return text

    def startTopicSport(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteSport()+' '
        #Ask the others about their favourite sport
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite sport?'
        elif rand==1:
            text+='Are you a sports fan?'
        elif rand==2:
            text+='Which sport are you interested in?'
        elif rand==3:
            text+='What is your preferred sport?'
        elif rand==4:
            text+='Which kind of sport do you like most?'
        print(name+text)
        return text

    def respondToSport(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteSport()
        print(text)

    def respond(self, message):
        split=message.split('. ')
        question=split[len(split)-1]
        question=question[:len(question)-1]
        guessedTopic=self.guessTopic(question)
        SWITCH_DICT={
            'sport':self.respondToSport,
            'music':self.respondToMusic,
            'job':self.respondToJob,
            'game':self.respondToGame,
            'food':self.respondToFood,
        }
        SWITCH_DICT.get(guessedTopic)()
        
    def guessTopic(self, text):
        bestTopic=None
        bestDistance=None
        words=text.split(' ')
        for word in words:
            for topic in topics:
                try:
                    distance=word2vecModel.distance(word, topic)
                    if bestTopic==None or distance<bestDistance:
                        bestTopic=topic
                        bestDistance=distance
                except:
                    continue
        return bestTopic
    
    def startTopic(self, doneTopics):
        openTopics=[topic for topic in topics if topic not in doneTopics]
        if len(openTopics)==0:
            return None
        rand=random.randint(0,len(openTopics)-1)
        if openTopics[rand]=='music':
            doneTopics.append('music')
            return self.startTopicMusic()
        elif openTopics[rand]=='job':
            doneTopics.append('job')
            return self.startTopicJob()
        elif openTopics[rand]=='food':
            doneTopics.append('food')
            return self.startTopicFood()
        elif openTopics[rand]=='game':
            doneTopics.append('game')
            return self.startTopicGame()
        elif openTopics[rand]=='sport':
            doneTopics.append('sport')
            return self.startTopicSport()
        else:
            return None

In [15]:
def generatePersons(nPersons):
    persons=[]
    for i in range(nPersons):
        person = Person()
        persons.append(person)
    return persons

In [46]:
def startChat(persons, nTopics):
    doneTopics=[]
    for person in persons: 
        person.introduce()
    for i in range(nTopics):
        personToStart=persons[random.randint(0, len(persons)-1)]
        message=personToStart.startTopic(doneTopics)
        if message==None:
            return
        random.shuffle(persons)
        for person in persons:
            if not person==personToStart:
                person.respond(message)

In [59]:
def startChatInteractive(persons):
    for person in persons: 
        person.introduce()
    while True:
        message=input('You: ')
        if message=='bye':
            return
        if not message=='':
            random.shuffle(persons)
            for person in persons:
                person.respond(message)

In [17]:
def generateDescriptions(persons):
    for person in persons:
        person.generateDescription()
        print('\n')

In [75]:
myPersons=generatePersons(4)

In [31]:
startChat(myPersons, 5)           

Hans:   Hey, I am Hans Beck, I'm 76 years old and I am from Barbados.
Charles:   Hey, my name is Charles Wright, I am 66 years old and I'm from Saint Lucia.
Adam:   Hey, I'm Adam Scott, I'm 53 years old and I come from Malaysia.
Liu:   Hello, I am Liu Xia, I'm 29 years old and I am from Argentina.
Charles:   I really enjoy playing Aeroplane Chess. Are there games that you like to play?
Liu:   My favourite game is Washington's War.
Hans:   I really enjoy playing Secret Wars II.
Adam:   I like In a Pickle.
Hans:   I really enjoy listening to narcocorrido. Which type of music do you enjoy most?
Liu:   I like ancient music.
Charles:   I really enjoy listening to Austropop.
Adam:   My favourite music genre is Italo dance.
Adam:   My favourite sport is bandy. What is your favourite sport?
Charles:   I like playing precision flying.
Liu:   I'm a fan of speed-ball.
Hans:   I like to play Classical dressage.
Adam:   I make a living from working as a apron supervisor. What do you work as?
Hans: 

In [61]:
startChatInteractive(myPersons)

Michel:   Hey, I'm Michel Bouvard, I'm 24 years old and I come from Eswatini.
Philip:   Hello, I'm Philip William, I am 34 years old and I live in Slovenia.
Richard:   Hi, I'm Richard Taylor, I am 34 years old and I come from Uganda.
Ludwig:   Good day, I'm Ludwig Traube, I am 42 years old and I'm from Brunei.
You: do you play soccer?
Michel:   I like to play Space Fleet.
Philip:   I like Karapan sapi.
Ludwig:   My favourite game is Shout About TV.
Richard:   My favourite game is Lexicon (game).
You: bye


In [20]:
generateDescriptions(myPersons)

C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Rico Rodriguez, who is from the country of Bolivia, has 18 years to be exact.
The Fliegel-Adjutant, Frankie Adjutant, is a key person at the company,Rico.
Taido is a favourite sport of the region of Rome.
The favourite music of Rome is the underground music.
The favourite religion of Rome is the Jewish religion.
The Top of the Order is the favourite football team of Rome.
The favourite food of Rome is the Imokilly Regato.
The collection of Australian timber samples in the form of books is a key interest in the book, Rome.


John Madden, who was born in the country of Moldova, has 40 years of age.
John's occupation is cupping therapy.
Bar billiards are the favourite sport of John.
The favourite of John is Minnesang.
The favourite building of John is the organ building.
The Left Hand of Eternity is John's favourite games.
Nasi Bogana is a favourite of John.
John Madin is interested in the Commemoration - Belgium.


Thomas Taylor, who is from the country of Syria, has 39 years to his cred

In [76]:
for person in myPersons:
    person.showProperties()

name: Aka / AKA / a.k.a.
age: 75
country: Argentina
book: tenderness
music: speed garage
job: party leader
sport: Laser-Run
film: Pawe i Gawe
game: Dystopian Wars
food: BP-5 Compact Food
object of interest: Tribute to Pierre Fermat

name: Walter Adams
age: 58
country: Nauru
book: disdain
music: neofolk
job: Kara-e mekiki
sport: summer skiing area
film: Mumford
game: Ringo
food: Black seabream
object of interest: 'Blue gums in a billabong' by Kerry and Co

name: Alan White
age: 29
country: Belarus
book: Ecophobia
music: cosmic disco
job: poultry seller
sport: kuningaspallo
film: The Matrix
game: BrikWars
food: Fried Rice with Salted Fish and Chicken
object of interest: Giraffe Head

name: Edward Grant
age: 46
country: Republic of the Congo
book: ferocity
music: spirituals
job: servant
sport: league
film: To Marry a Captain
game: Blacks & Whites
food: Crispy Crunch
object of interest: Tribute to Corrado Ricci



In [41]:
def a():
    text=input('input: ')
    if text=='':
        print('a')

In [42]:
a()

input: 
a
